# find-about-pages-v2
SanjayKAroraPhD@gmail.com <br>
December 2018

## Description
Most of the interesting company information is not on home pages but on about us pages.  This script finds those links from the home pages for further crawling. 

## Change log
v3 adds the python boilerplate  api for web page cleaning.

## TODO:
* Need to make better use of all pages in the site, e.g., to improve data quality and use additional paragraph data found on non-homepages 
* Standardize firm name cleaning code utility into a universally accessible module
* Train a more effective boilerplate model to capture meaningful text from firm web pages

In [123]:
# import data processing and other libraries
import csv
import sys
import requests
import os
import re
import pprint
import pymongo
import traceback
from time import sleep
import requests
import pandas as pd
import io
from IPython.display import display
import time
import numpy as np
from bs4 import BeautifulSoup
import string
from urllib.parse import urlparse, urljoin
import requests

In [7]:
from boilerpipe.extract import Extractor

In [8]:
MONGODB_DB = "FirmDB_20181203"
MONGODB_COLLECTION = "pages_depth0"
CONNECTION_STRING = "mongodb://localhost"
username = "scrapy"
password = "eager"
authSource = "FirmDB"
authMechanism='SCRAM-SHA-1'

client = pymongo.MongoClient(CONNECTION_STRING, username=username, password=password, authSource=authSource, authMechanism=authMechanism)
db = client[MONGODB_DB]
col = db[MONGODB_COLLECTION]

DATA_DIR = '/Users/sarora/dev/EAGER/data/orgs/depth0_boilerpipe/'

In [174]:
# gather unique firm_names from mongodb

def get_firm_aggregates ():
    query = [ { "$group": {"_id":"$firm_name" , "number":{"$sum":1}} } ]
    # test for a specific firm
    # query = [ {"$match":{"firm_name": 'Vizio Inc.'}}, { "$group": {"_id":"$firm_name" , "number":{"$sum":1}} } ]
    results = col.aggregate(query)

    mongo_dict = {}
    for result in results:
        key = (result['_id'])
        if key:
            mongo_dict[key[0]] = result['number']
    
    return mongo_dict

results_dict = get_firm_aggregates()
firm_names = results_dict.keys()
print (len(firm_names))
pp = pprint.PrettyPrinter()
pp.pprint(firm_names)

1152
dict_keys(['Xintec Inc.', 'W&Wsens Devices', '22nd Century Limited', 'Yamaha Corporation', 'ZENA TECHNOLOGIES', 'Wikipad', 'Wine.com', 'Vizio Inc.', 'WHATSAPP INC.', 'VMware', 'VEECO PRECISION SURFACE PROCESSING LLC', 'WestPoint Home', 'VIASAT INC.', 'DuPont Teijin Films U.S. Limited Partnership', 'Vaxiion Therapeutics', 'Silicon Space Technology Corp.', 'Universal Leaf Tobacco Co.', 'TRIDONIC GMBH & CO KG', 'Valicor', 'ZON', 'Transitions Optical', 'Kabushikikaisha Toshiba', 'Waters Technologies Corporation', 'Virent', 'Nederlandse Organisatie voor toegepast-natuurwetenschappelijk onderzoek TNO', 'Thorn Bioscience LLC', 'Xerox Corporation', 'Metabolix', 'Vascular BioSciences', 'Wizard Labs', 'Casio Computer Co.', 'Crystal Solar Inc.', 'WOVN', 'Wireless Environment', 'Hamilton Sundstrand Corporation', 'Ube Industries', 'UChicago Argonne', 'Weatherford Canada Partnership', 'Vorbeck Materials Corporation', 'Wenger Corporation', 'TECNIUM', 'Wave Energy Conversion Corporation of Americ

In [182]:
# find all links that have about, company, and corporate mentions 
# 1. pull down home page
# 2. get a list of all links in Beautiful Soup
# 3. keep only those that match a regex

firm_about_dict = {}
firm_no_about_dict = {}

def fix_url(url, home_url, domain):
    return urljoin (home_url, url, allow_fragments=False)

def get_domain (url):
    o = urlparse(url)
    domain = o.netloc
    return domain

def get_link_path (url):
    o = urlparse(url)
    path = o.path + o.params + o.query + o.fragment
    return path

def get_about_links_from_home_pages (): 
    for firm_name in firm_names: 
        regex = '^' + re.escape(firm_name) + '$'
        results = col.find( {"firm_name": re.compile(regex, re.IGNORECASE)}).sort("depth",pymongo.ASCENDING) 

        result = results[0]
        html = result['html'][0]
        home_url = result['url'][0]
        domain = get_domain(home_url).strip('\/')
        
        # print ("Working on " + firm_name + ' at ' + home_url)
        soup = BeautifulSoup(html, 'lxml')
        regex_in = re.compile('(about|company|corporate|who.we.are|(^|/)vision|profile|corporate|management|team|history|values)', flags=re.IGNORECASE)
        abouts = {a.get('href'): ' '.join(str(a.string).split()) for a in soup.find_all('a', href=True) 
                  if regex_in.search(get_link_path(a.get('href'))) or  regex_in.search(str(a.string))}
        
        abouts_fixed = {fix_url(link, home_url, domain): text for link, text in abouts.items()} 
 
        abouts_clnd = {link: text for link, text in abouts_fixed.items() if re.search(domain, link)}
        regex_out = re.compile('(order|sale|api|product|company.?home|pdf$)', flags=re.IGNORECASE)
        abouts_clnd = {link: text for link, text in abouts_clnd.items() 
                       if not regex_out.search(link) and not regex_out.search(text)}
        
        # pp.pprint (abouts_clnd)
        
        if abouts_clnd:
            firm_about_dict[firm_name] = abouts_clnd
        else: 
            firm_no_about_dict[firm_name] = home_url

get_about_links_from_home_pages()

In [139]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.10; rv:62.0) Gecko/20100101 Firefox/62.0'}
timeout = 5

def is_valid_url (url):
    try:
        resp = requests.get(url, headers=headers, timeout=timeout, verify=False)
        # print ("\tStatus code: " + str(resp.status_code))
        if resp.status_code >= 200 and resp.status_code < 400:
            return True
        else:
            return False
    except Exception as e: 
        print(e)
        return False
        # traceback.print_exc()

In [140]:
for firm_key, abouts in firm_about_dict.items():
    print (firm_key)
    sorted_links_by_len = sorted(abouts, key=len)
    has_about_been_found = False
    pp.pprint(sorted_links_by_len)

Xintec Inc.
['http://www.xintec.com.tw/eng/AX_Introduction.aspx',
 'http://www.xintec.com.tw/eng/AX_Management-Team.aspx',
 'http://www.xintec.com.tw/eng/IR_Company_Profile.aspx',
 'http://www.xintec.com.tw/eng/AX_Core-Value_Vision.aspx',
 'http://www.xintec.com.tw/eng/CSR_Corporate_Governance.aspx',
 'http://www.xintec.com.tw/eng/CSR_WESC_Employee_Profiles.aspx']
W&Wsens Devices
['https://www.wwsensdevices.com/#comp-iocl6fjb']
22nd Century Limited
['http://ir.xxiicentury.com/',
 'http://www.xxiicentury.com/#',
 'https://youtu.be/S24mKzpasIE',
 'https://youtu.be/JKb0WmD7RSc',
 'http://www.xxiicentury.com/profile/',
 'http://www.xxiicentury.com/history/',
 'http://www.xxiicentury.com/our-management/',
 'http://ir.xxiicentury.com/corporate-governance',
 'http://buffalonews.com/2017/04/06/high-hopes-22nd-century-group/',
 'http://wivb.com/2017/09/21/clarence-based-company-stands-alone-on-future-of-tobacco/']
Yamaha Corporation
['http://www.yamahaboats.com/series',
 'http://yamahawaverunne

 'http://www.shinetsu.co.jp/en/company/movie.html',
 'http://www.shinetsu.co.jp/en/company/profile.html',
 'http://www.shinetsu.co.jp/en/company/history.html',
 'http://www.shinetsu.co.jp/en/company/network.html',
 'http://www.shinetsu.co.jp/en/company/quality.html',
 'http://www.shinetsu.co.jp/en/company/supplier_hotline.html']
UOP LLC
['https://www.phoenix.edu/about_us/title-ix.html',
 'https://www.phoenix.edu/about_us/trademark.html',
 'https://www.phoenix.edu/about_us/contact_us.html',
 'https://www.phoenix.edu/about_us/regulatory.html',
 'https://www.phoenix.edu/about_us/media-center.html',
 'https://www.phoenix.edu/about_us/accreditation.html',
 'https://www.phoenix.edu/about_us/campus-safety.html',
 'https://www.phoenix.edu/about_us/code-of-ethics.html',
 'http://www.apollo.edu/about/accessibility-statement.html',
 'https://www.phoenix.edu/about_us/media-center/leadership.html',
 'https://www.phoenix.edu/about_us/about_university_of_phoenix.html',
 'https://www.phoenix.edu/about

 'https://monsanto.com/company/',
 'https://monsanto.com/company/media/',
 'https://monsanto.com/company/history/',
 'https://monsanto.com/company/outreach/',
 'https://monsanto.com/company/locations/',
 'https://monsanto.com/company/governance/',
 'https://monsanto.com/company/contact-us/',
 'https://monsanto.com/company/commitments/',
 'https://monsanto.com/company/procurement/',
 'https://monsanto.com/company/time-to-grow/',
 'https://www.investor.bayer.de/en/overview/',
 'https://monsanto.com/company/sustainability/',
 'https://www.bayer.com/en/products-overview.aspx',
 'https://monsanto.com/company/monsanto-growth-ventures/',
 'https://monsanto.com/company/partnering-and-licensing/',
 'https://monsanto.com/products/product-stewardship/technology-use-guides/#',
 'https://monsanto.com/products/product-stewardship/corn-rootworm-management/',
 'https://monsanto.com/products/product-stewardship/best-management-practices/',
 'https://monsanto.com/products/product-stewardship/insect-resi

['https://www.hpe.com/us/en/about.html',
 'https://www.hpe.com/us/en/about/jobs.html',
 'https://www.hpe.com/us/en/about/diversity.html',
 'https://www.hpe.com/us/en/servers/management.html',
 'https://www.hpe.com/us/en/networking/management.html',
 'https://www.hpe.com/us/en/about/legal/terms-of-use.html',
 'https://www.hpe.com/us/en/about/accessibility-aging.html',
 'https://www.hpe.com/us/en/integrated-systems/software.html',
 'https://www.hpe.com/us/en/storage/management-automation.html',
 'https://www.hpe.com/us/en/about/executive-briefing-centers.html',
 'https://www.hpe.com/us/en/networking/smb-networking-overview.html',
 'https://www.hpe.com/us/en/solutions/cloud/hybrid-it-management.html']
Hunt Energy Enterprises LLC
['http://www.huntoil.com',
 'http://www.huntrefining.com',
 'http://www.huntconsolidated.com/about.aspx',
 'http://huntenergyenterprises.com/about.aspx']
Pendar Technologies
['http://www.pendar.com/#about',
 'http://www.pendar.com/about.html',
 'http://www.pendar.

 'https://www.quest.com/solutions/microsoft-platform-management/',
 'https://support.quest.com/professional-services-product-select',
 'https://www.quest.com/products/foglight-for-storage-management/',
 'https://www.quest.com/products/kace-systems-management-appliance/',
 'https://www.quest.com/database-management/performance-monitoring/',
 'https://support.quest.com/consulting-services/database-management',
 'https://www.quest.com/products/desktop-authority-management-suite/',
 'https://www.quest.com/community/quest/endpoint-systems-management/',
 'https://www.quest.com/products/kace-cloud-mobile-device-management/',
 'https://www.oneidentity.com/solutions/privileged-access-management/',
 'https://www.quest.com/community/quest/microsoft-platform-management',
 'https://www.quest.com/solutions/performance-monitoring/#capabilities',
 'https://www.oneidentity.com/solutions/identity-and-access-management/',
 'https://www.quest.com/solutions/endpoint-systems-management/#platforms',
 'https:

 'https://amkor.com/technology/system-in-package/',
 'https://c44f5d406df450f4a66b-1b94a87d576253d9446df0a9ca62e142.ssl.cf2.rackcdn.com/2018/08/PHX-Business-Journal-2018-Corporate-Anniversaries-Amkor.pdf']
Banpil Photonics
['http://banpil.com/vmv.htm', 'http://banpil.com/comp_overview.htm']
Biocon Limited
['http://biocon.com/biocon_aboutus.asp',
 'http://biocon.com/biocon_aboutus_factsheet.asp',
 'http://biocon.com/biocon_aboutus_businesses.asp',
 'http://www.biocon.com/biocon_aboutus_biomalaysia.asp',
 'http://biocon.com/220918_Biocon_Company_Statement_SE.asp',
 'http://biocon.com/biocon_press_kmprofile.asp?subLink=down',
 'http://www.biocon.com/301118_Biocon_Company_Statement_SE.asp',
 'http://biocon.com/biocon_press_media_bioprofile.asp?subLink=down']
CLEARSIGN COMBUSTION CORPORATION
['http://ir.clearsign.com/overview',
 'https://www.clearsign.com/our-team/',
 'https://www.clearsign.com/applications/duplex-products-for-steam-generation/',
 'https://ir.clearsign.com/2018-10-01-ClearS

 'https://www.baesystems.com/en-us/our-company/inc-businesses/electronic-systems/employee-information']
Parion Sciences
['http://parion.com/about/',
 'http://parion.com/about/our-team/',
 'http://parion.com/about/board-of-directors/',
 'http://parion.com/news-and-events/parion-sciences-receives-2015-life-sciences-award-as-best-late-stage-product-development-company/']
Boston Scientific Scimed
['http://www.bostonscientific.com/en-US/about-us.html',
 'http://www.bostonscientific.com/en-US/about-us/awards.html',
 'http://www.bostonscientific.com/en-US/about-us/history.html',
 'http://www.bostonscientific.com/en-US/about-us/ventures.html',
 'http://www.bostonscientific.com/en-US/about-us/locations.html',
 'http://www.bostonscientific.com/en-US/about-us/who-we-are.html',
 'http://www.bostonscientific.com/en-US/about-us/leadership.html',
 'http://www.bostonscientific.com/en-US/corporate-citizenship.html',
 'http://www.bostonscientific.com/en-US/about-us/core-businesses.html',
 'http://www.bo

In [82]:
final_about_dict = {}
for firm_key, abouts in firm_about_dict.items():
    sorted_links_by_len = sorted(abouts, key=len)
    has_about_been_found = False
    pp.pprint(sorted_links_by_len)
    for link in sorted_links_by_len:
        if is_valid_url (link):
            has_about_been_found = True
            final_about_dict[firm_key] = link
            break
    if not has_about_been_found: # same as continuing outerloop if about page has been found
        home = get_domain(sorted_links_by_len[0])
        firm_no_about_dict[firm_key] = home
        final_about_dict[firm_key] = home

In [83]:
# check again since some firms may have been added 
pp.pprint (firm_no_about_dict)

{'Dymax Corporation': 'https://dymax.com/'}


In [84]:
# write about pages to file
pp.pprint(final_about_dict)

{}
